# Agenda

- Generator functions
- Generator comprehensions
- Concurrency
    - Threads
    - Multiprocessing
    - `asyncio`

In [1]:
def myfunc(x, y):
    for one_number in range(x, y):
        yield one_number

In [2]:
myfunc(10, 20)

<generator object myfunc at 0x10ce49700>

In [3]:
for one_item in myfunc(10, 20):
    print(one_item)

10
11
12
13
14
15
16
17
18
19


In [4]:
def evenrange(start, finish):
    for one_number in range(start, finish):
        if one_number % 2:
            continue
        yield one_number

In [5]:
for one_item in evenrange(10, 20):
    print(one_item)

10
12
14
16
18


# Things to remember about generators

- `yield` means: return the value (in the iteration/loop), and go to sleep right after
- if we use `return` without an argument, we effectively raise `StopIteration`
- Also when the function exits normally/naturally, we effectively raise `StopIteration`
- Don't raise `StopIteration` in a generator function! Let the system do it for you
- Local variables retain their values between invocations of `yield`

In [6]:
def myfunc():
    return f'Hello, world!'

In [7]:
def mygen():
    yield f'Hello, world!'

In [9]:
myfunc()

'Hello, world!'

In [10]:
mygen()

<generator object mygen at 0x10cebfd70>

In [11]:
type(myfunc)

function

In [12]:
type(mygen)

function

In [13]:
myfunc.__code__.co_consts

(None, 'Hello, world!')

In [14]:
myfunc.__code__.co_flags

3

In [16]:
bin(myfunc.__code__.co_flags)

'0b11'

In [17]:
bin(mygen.__code__.co_flags)

'0b100011'

In [18]:
import dis

In [19]:
dis.show_code(myfunc)

Name:              myfunc
Filename:          /var/folders/rr/0mnyyv811fs5vyp22gf4fxk00000gn/T/ipykernel_76740/2896535200.py
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  0
Stack size:        1
Flags:             OPTIMIZED, NEWLOCALS
Constants:
   0: None
   1: 'Hello, world!'


In [20]:
dis.show_code(mygen)

Name:              mygen
Filename:          /var/folders/rr/0mnyyv811fs5vyp22gf4fxk00000gn/T/ipykernel_76740/3368138942.py
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  0
Stack size:        1
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR
Constants:
   0: None
   1: 'Hello, world!'


In [21]:
def myfunc():
    return 1
    return 2
    return 3

In [22]:
dis.dis(myfunc)

  1           0 RESUME                   0

  2           2 LOAD_CONST               1 (1)
              4 RETURN_VALUE


In [23]:
def myfunc():
    if False:
        yield 'Hello'
    

In [24]:
dis.show_code(myfunc)

Name:              myfunc
Filename:          /var/folders/rr/0mnyyv811fs5vyp22gf4fxk00000gn/T/ipykernel_76740/1059155119.py
Argument count:    0
Positional-only arguments: 0
Kw-only arguments: 0
Number of locals:  0
Stack size:        1
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR
Constants:
   0: None


In [25]:
g1 = evenrange(10, 20)
g2 = evenrange(10, 20)
g3 = evenrange(10, 20)


In [26]:
next(g1)

10

In [27]:
next(g1)



12

In [28]:
next(g1)

14

In [29]:
g1.gi_frame.f_lineno

5

In [30]:
g2.gi_frame.f_lineno

1

# Exercise: `read_n`

When we iterate over a file, each iteration gives us the next line, up to and including `\n`. Sometimes, though, we want to read a file in chunks of 2, 5, or 10 lines.

1. Write `read_n`, that takes two arguments:
    - The name of a file
    - The number of lines we want to get back in each iteration
2. With each iteration, we should get that number of lines back, as a single string.
3. The final iteration can give us fewer lines.

Example:

```python
for one_chunk in read_n('/etc/passwd', 5):
     print(one_chunk) 
```

In [ ]:
def read_n(filename, n):
    